In [54]:
import numpy as np
from shapely.geometry import LineString, Polygon
import geopandas as gpd

In [ ]:
# Read in linestring object
file = '/Users/hyin/usgs_mendenhall/ffsimmer/fault-extrusion/myanmar-example/myanmar-trace.shp'
zmax = 20.0 # maximum depth for extrusion (km)
dip_deg = 80 # dip angle in degrees

trace = gpd.read_file(file)
trace.geometry

## Extract the individual segments from each linestring
line = trace.geometry.values[0] # assumes there's only one linestring in the shapefile
coords = list(line.coords)
print(coords)

[(96.74163552091174, 25.63630875316376), (96.09950517413058, 24.298537197369676), (96.72172416054114, 20.286549291978623)]


In [62]:
def strike_and_dipdir(p0, p1):
    '''
    Calculate strike and dip direction from two points using the right-hand rule.
    '''
    dx = p1[0] - p0[0]
    dy = p1[1] - p0[1]
    strike = np.degrees(np.arctan2(dx, dy)) % 360
    dipdir = (strike + 90) % 360
    return strike, dipdir


In [63]:
segment_data = []   # Initialize a list of segments with their extrusion vectors

for i in range(len(coords) - 1):
    start = coords[i]
    end = coords[i + 1]

    print(f"Segment {i}: start={start}, end={end}")

    strike, dipdir = strike_and_dipdir(start, end)
    print(f"Strike: {strike:.2f} degrees")
    print(f"Dip Direction: {dipdir:.2f} degrees")

    # Convert the dip direction to a unit vector in the horizontal plane (east, north)
    dip_rad = np.radians(dipdir)
    dip_xy = np.array([np.sin(dip_rad), np.cos(dip_rad)]) # east, north
    dip_xy /= np.linalg.norm(dip_xy)    # normalize to unit vector to get dip direction unit vector
    print(f"Dip direction unit vector (dip_xy): {dip_xy}")
    L_h = zmax / np.tan(np.radians(dip_deg))
    print(f"Horizontal extrusion length (L_h): {L_h:.2f} km")
    
    V = np.array([
        dip_xy[0] * L_h,
        dip_xy[1] * L_h,
        -zmax
    ])
    print(f"Extrusion vector (V): {V}\n")

    segment_data.append({
    "start": start,
    "end": end,
    "V": V
    })

# example of how to access the data for the first segment
segment_data[0]['start']        # each segment has 'start', 'end', and 'V' keys

Segment 0: start=(96.74163552091174, 25.63630875316376), end=(96.09950517413058, 24.298537197369676)
Strike: 205.64 degrees
Dip Direction: 295.64 degrees
Dip direction unit vector (dip_xy): [-0.90152306  0.43273107]
Horizontal extrusion length (L_h): -3.53 km
Extrusion vector (V): [  3.17925678  -1.52604325 -20.        ]

Segment 1: start=(96.09950517413058, 24.298537197369676), end=(96.72172416054114, 20.286549291978623)
Strike: 171.18 degrees
Dip Direction: 261.18 degrees
Dip direction unit vector (dip_xy): [-0.98818625 -0.15325775]
Horizontal extrusion length (L_h): -3.53 km
Extrusion vector (V): [  3.48487795   0.54046953 -20.        ]



(96.74163552091174, 25.63630875316376)

## Deal with the extrusion conflicts at depth
For each "vertex" (intersection between two line segments) we need to find a common point at depth where the two planes can intersect

In [35]:
coords[0]

(96.74163552091174, 25.63630875316376)

In [64]:
P_bottom = {}

# Calculate the first vertex
P0 = np.array([coords[0][0], coords[0][1], 0.0])    # Make surfaace point 3D by adding z=0
P_bottom[0] = P0 + segment_data[0]["V"]

# Calculate the middle vertices
for i in range(1, len(coords) - 1):
    Pi = np.array([*coords[i], 0.0])

    V_left  = segment_data[i - 1]["V"]
    V_right = segment_data[i]["V"]

    P_left  = Pi + V_left
    P_right = Pi + V_right

    P_bottom[i] = 0.5 * (P_left + P_right)

# Calculate the last vertex
Pn = np.array([coords[-1][0], coords[-1][1], 0.0])
P_bottom[len(coords)-1] = Pn + segment_data[-1]["V"]

P_bottom

{0: array([ 99.9208923,  24.1102655, -20.       ]),
 1: array([ 99.43157254,  23.80575034, -20.        ]),
 2: array([100.20660211,  20.82701882, -20.        ])}

# Build subfault polygons

In [65]:
polys = []

for i in range(len(coords) - 1):
    P0 = np.array(coords[i])
    P1 = np.array(coords[i + 1])

    B0 = P_bottom[i]
    B1 = P_bottom[i + 1]

    poly = Polygon([
        (P0[0], P0[1], 0.0),
        (P1[0], P1[1], 0.0),
        (B1[0], B1[1], B1[2]),
        (B0[0], B0[1], B0[2]),
        (P0[0], P0[1], 0.0)
    ])

    polys.append(poly)

polys

[<POLYGON Z ((96.742 25.636 0, 96.1 24.299 0, 99.432 23.806 -20, 99.921 24.11...>,
 <POLYGON Z ((96.1 24.299 0, 96.722 20.287 0, 100.207 20.827 -20, 99.432 23.8...>]

In [66]:
gdf_out = gpd.GeoDataFrame(
    geometry=polys,
    crs=4326).to_crs("EPSG:4326")

gdf_out.to_file("fault_surface_segments.geojson", driver="GeoJSON")